In [1]:
import pickle
with open('./hw1_data/LFW_DATA.pickle','rb') as f:
    data = pickle.load(f)
#print data.keys()
#[u'database_name', u'query_identity', u'database_identity', u'query_name', u'database_feature', u'query_feature']
src = data['database_feature']


In [4]:
import spams
import numpy as np
print(src.shape)
num = src.shape[0]
X = src.reshape([num, 80,59])

X = np.swapaxes(X,axis1=0,axis2=2)

lambda1 = [10**i for i in range(-6,-3,1)]
K = range(100,3200,400)
param = { 'K' : 400, # learns a dictionary with 100 elements
          'lambda1' : 10**-2, 'numThreads' : -1, 'mode':2,
          'iter' : 1000}

    
print(X.shape)




(13113, 4720)
(59, 80, 13113)


In [3]:
import scipy
import scipy.spatial
import numpy as np
import spams
# print D
pathces_num = 80
test = data['query_feature']
num = test.shape[0]
qs = test.reshape([num, 80,59])
qs = np.swapaxes(qs,axis1=0,axis2=2)
# 
with open('./dict2.npy','rb') as f:
    D_list = np.load(f)
    D_list = [np.asfortranarray(D) for D in D_list]
def get_codeword(D_list, X,lambda1=0.01):
    
    param = { 'lambda1' : lambda1, 'lambda2' : 0,'numThreads' : -1, 'mode':2}
    buf = []
    for i in range(pathces_num):
        q = np.asfortranarray(X[:,i,:])
        D = D_list[i]
        a = spams.lasso(q,D=D,return_reg_path = False,**param)
        buf.append(a)
    buf = scipy.sparse.vstack(buf)
    codeword = np.zeros(buf.shape, dtype=np.bool)
    codeword[buf.nonzero()] = True
    return codeword

#
num = src.shape[0]
X = src.reshape([num, 80,59])

X = np.swapaxes(X,axis1=0,axis2=2)
print(X.shape)
# 
q_codeword = get_codeword(D_list, qs)
codeword = get_codeword(D_list, X)
print(q_codeword.shape)
print(codeword.shape)


(59, 80, 13113)
(32000, 120)
(32000, 13113)


In [6]:
import numpy as np
# from sklearn.metrics import average_precision_score

# 注意要計算and而不是xor(hamming distance)
def similarity_sparse_coding(codeword, query):
    q = query.ravel()
    
    distances = []
    for x in codeword:
#         distances.append(scipy.spatial.distance.hamming(x,query))
        distances.append(np.sum(np.logical_and(x,q)))
    return reversed(np.argsort(distances))


database = np.swapaxes(codeword, axis1=0, axis2=1)
labels = data['database_name'].ravel()

queries = np.swapaxes(q_codeword, axis1=0, axis2=1)
query_labels = data['query_name'].ravel()
print(database.shape, queries.shape)


import time
t1 = time.time()
calculate_map(database, labels, queries, query_labels, similarity_sparse_coding)
# mAP :  0.13721717999431532
# mAP : 0.1581216311119271 with 800
print('cost time : ', time.time() - t1)

(13113, 32000) (120, 32000)


NameError: name 'mAp' is not defined

In [ ]:
# inverted index for ranking
import threading
import numpy as np
# input : database[doc_num, word_dim]
# output : inverted_list[word_dim] each entry contains a list for docs index.
def generate_inverted_list(database, thread_count = 16):
    assert len(database.shape) == 2
    num, dim = database.shape
    print('generating inverted list with %d doc, %d interested words.' % (num, dim))
    inv_list = [[] for i in range(dim)]
    
    for idx,x in enumerate(database):
        for i,j in enumerate(x):
            if j == True:
                inverted_list[i].append(idx)
    return inv_list
#output : return an idx_list with decresing similarity which depends on occurrence count in inverted list
database = np.swapaxes(codeword, axis1=0, axis2=1)
print(database.shape)

import time
t1 = time.time()
inv_list = generate_inverted_list(database)
t2 = time.time()
print('takes : ', t2-t1)

    

(13113, 64000)
generating inverted list with 13113 doc, 64000 interested words.


In [77]:
import pickle

def rank_by_inverted_list(database, inverted_list, query):
    assert len(query.shape) == 1
    assert len(database.shape) == 2
    num, _ = database.shape
    count = [0] * num
    for i,j in enumerate(query):
        if j == True:
            for idx in inverted_list[i]:
                count[idx] += 1
    return reversed(np.argsort(count))
# print inv_list
database = np.swapaxes(codeword, axis1=0, axis2=1)
labels = data['database_name'].ravel()

queries = np.swapaxes(q_codeword, axis1=0, axis2=1)
query_labels = data['query_name'].ravel()

def similarity_with_inverted_list(codeword, query):
    return rank_by_inverted_list(codeword, inv_list, query)


calculate_map(database, labels, queries, query_labels, similarity_with_inverted_list)
# with open('inv_list.pickle','wb') as f:
#     pickle.dump(inv_list, f, protocol=2)


(13113, 32000) (120, 32000)
Done


In [4]:
from l2_distance import l2_distance #l2_distance.py
from sparse_coding import sparse_coding,similarity_sparse_coding #sparse_coding.py
from calculate_map import calculate_map #calculate_map.py
import pickle
lfw_path = './hw1_data/LFW_DATA.pickle'
with open(lfw_path, 'rb') as f:
    lfw = pickle.load(f)
database = lfw['database_feature']
queries = lfw['query_feature']
labels = lfw['database_name'].ravel()
query_labels = lfw['query_name'].ravel()

#part 1: l2_distance and calculate_map
# mAP = calculate_map(database, labels, queries, query_labels, l2_distance)
# print(mAP)

#part 2: sparse_coding
param = { 'K' : 400,
                  'lambda1' : 0.01, 'numThreads' : -1, 'mode':2,
                  'iter' : -10}
        
q_sparse, db_sparse = sparse_coding(lfw['query_feature'], lfw['database_feature'], param) #build sparse dict and lookup using spams library
sparse_map = calculate_map(db_sparse, labels, q_sparse, query_labels, similarity_sparse_coding)# results_sparse = distance(q_sparse, db_sparse) #you can use l2_distance in part1 or try any distance metric, like cos, l1 
print(sparse_map)
#part 3
lambda1 = [10**i for i in range(-6,-3,1)]
K = range(100,3200,400)
iter_ = -10
for k in K:
    for l1 in lambda1:
        

        param = { 'K' : k,
                  'lambda1' : l1, 'numThreads' : -1, 'mode':2,
                  'iter' : iter_}
        q_sparse, db_sparse = sparse_coding(lfw['query_feature'], lfw['database_feature'], param) #build sparse dict and lookup using spams library
        sparse_map = calculate_map(db_sparse, labels, q_sparse, query_labels, similarity_sparse_coding)# results_sparse = distance(q_sparse, db_sparse) #you can use l2_distance in part1 or try any distance metric, like cos, l1 

        print(k,l1, sparse_map)







train


In [ ]:
sparse_map = calculate_map(db_sparse, labels, q_sparse, query_labels, similarity_sparse_coding)# results_sparse = distance(q_sparse, db_sparse) #you can use l2_distance in part1 or try any distance metric, like cos, l1 
print(sparse_map)
